## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-04-16-51-00 +0000,wsj,Trump Threatens to Block SNAP Payments During ...,https://www.wsj.com/politics/policy/trump-thre...
1,2025-11-04-16-46-19 +0000,nypost,Victoria’s Secret investor urges ouster of ‘ov...,https://nypost.com/2025/11/04/business/victori...
2,2025-11-04-16-44-00 +0000,wsj,"Shopify Revenue Tops Views, Guides to Strong H...",https://www.wsj.com/business/earnings/shopify-...
3,2025-11-04-16-41-01 +0000,nyt,France Threatens to Ban Shein Over Sale of ‘Ch...,https://www.nytimes.com/2025/11/03/business/sh...
4,2025-11-04-16-38-35 +0000,bbc,Badenoch calls for sackings at BBC over edited...,https://www.bbc.com/news/articles/ckgzkyk7knzo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2328/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,43
40,new,20
84,will,17
67,election,14
355,dies,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
150,2025-11-04-10-00-00 +0000,wsj,When the Supreme Court hears arguments over Pr...,https://www.wsj.com/politics/policy/whats-at-s...,127
235,2025-11-03-22-40-36 +0000,nypost,Trump tells New York ‘you must vote’ for Andre...,https://nypost.com/2025/11/03/us-news/trump-te...,123
152,2025-11-04-09-30-53 +0000,bbc,Trump backs Cuomo for New York City mayor and ...,https://www.bbc.com/news/articles/c231e284345o...,113
197,2025-11-04-02-00-00 +0000,wsj,Some Tuesday elections will test whether Trump...,https://www.wsj.com/politics/elections/divided...,98
305,2025-11-03-18-56-35 +0000,bbc,Trump administration will tap emergency fund t...,https://www.bbc.com/news/articles/cglg632n719o...,91


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
150,127,2025-11-04-10-00-00 +0000,wsj,When the Supreme Court hears arguments over Pr...,https://www.wsj.com/politics/policy/whats-at-s...
235,76,2025-11-03-22-40-36 +0000,nypost,Trump tells New York ‘you must vote’ for Andre...,https://nypost.com/2025/11/03/us-news/trump-te...
95,61,2025-11-04-11-52-00 +0000,wsj,"Dick Cheney, Powerful Former Vice President Wh...",https://www.wsj.com/politics/dick-cheney-power...
298,55,2025-11-03-19-16-00 +0000,wsj,Federal Reserve governor Lisa Cook said she su...,https://www.wsj.com/economy/central-banking/em...
19,44,2025-11-04-15-38-36 +0000,nypost,NJ Election Day bomb threats shut down polling...,https://nypost.com/2025/11/04/us-news/nj-elect...
281,42,2025-11-03-20-07-00 +0000,wsj,The U.S. said it would use emergency funds to ...,https://www.wsj.com/politics/policy/u-s-to-pay...
29,40,2025-11-04-15-01-47 +0000,nypost,France threatens to block Shein over sale of c...,https://nypost.com/2025/11/04/world-news/franc...
197,38,2025-11-04-02-00-00 +0000,wsj,Some Tuesday elections will test whether Trump...,https://www.wsj.com/politics/elections/divided...
34,37,2025-11-04-14-50-51 +0000,bbc,Reeves' pre-Budget speech fails to rule out ta...,https://www.bbc.com/news/articles/ce8g6kdgzelo...
257,36,2025-11-03-21-39-00 +0000,wsj,"Hundreds of travel businesses, including casin...",https://www.wsj.com/politics/policy/travel-ind...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
